In [1]:
import Graphics.EasyPlot
import Database.HDBC
import Database.HDBC.Sqlite3

In [2]:
db <- connectSqlite3 "../data/stocks.sqlite3"

In [3]:
aaplRaw <- quickQuery db "SELECT adjclose FROM aapl" []

In [4]:
:t aaplRaw

aaplRaw :: [[SqlValue]]

In [5]:
aapl = map (fromSql . head) aaplRaw :: [Double]

In [12]:
plot X11 $ zip [0,-1..] aapl

True

In [13]:
plot X11 $ Data2D [Style Lines] [] (zip [0,-1..] aapl)

True

In [14]:
movingSum :: [Double] -> [Double] -> [Double]
movingSum xs [] = [sum xs]
movingSum (x:xs) (y:ys) = (sum (x:xs)):movingSum (xs++[y]) ys

In [15]:
movingSum [1,2,3] [4,5,6,7,8,9,10]

[6.0,9.0,12.0,15.0,18.0,21.0,24.0,27.0]

In [16]:
movingAverage :: [Double] -> Int -> [Double]
movingAverage xs windowSize
 | windowSize < 1 = error "Cannot have non-positive window size."
 | windowSize > length xs = []
 | otherwise = map (/ fromIntegral windowSize) movingSums
 where
    movingSums = movingSum (take windowSize xs) (drop windowSize xs)

In [17]:
movingAverage [1..10] 3

[2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0]

In [18]:
aaplMA = movingAverage aapl 200

In [21]:
plot X11 $ Data2D [Style Lines] [] (zip [0,-1..] aaplMA)

True

In [22]:
plot X11 [Data2D [Style Lines] [] (zip [0,-1..] aapl), Data2D [Style Lines] [] (zip [0,-1..] aaplMA)]

True

In [23]:
plot X11
   [ Data2D [Style Lines] [] (zip [0,-1..] (take 2000 aapl))
   , Data2D [Style Lines] [] (zip [0,-1..] (take 2000 aaplMA))
   ]

True

In [24]:
plot X11
   [ Data2D [Style Lines] [] (zip [0,-1..] (take 2000 aapl))
   , Data2D [Style Lines] [] (zip [-100,-101..] (take 2000 aaplMA))
   ]

True

In [25]:
plot X11
   [ Data2D [Style Lines, Title "AAPL Adj. Close"] [] (zip [0,-1..] (take 2000 aapl))
   , Data2D [Style Lines, Title "AAPL 200-day MA"] [] (zip [-100,-101..] (take 2000 aaplMA))
   ]

True

In [26]:
plot X11
   [ Data2D [Style Lines, Title "AAPL Adj. Close", Color Blue] [] (zip [0,-1..] (take 2000 aapl))
   , Data2D [Style Lines, Title "AAPL 200-day MA", Color Black] [] (zip [-100,-101..] (take 2000 aaplMA))
   ]

True

In [27]:
plot (PNG "adjcloseWith200DayMA.png")
   [ Data2D [Style Lines, Title "AAPL Adj. Close", Color Blue] [] (zip [0,-1..] (take 2000 aapl))
   , Data2D [Style Lines, Title "AAPL 200-day MA", Color Black] [] (zip [-100,-101..] (take 2000 aaplMA))
   ]

True

In [36]:

googlRaw <- quickQuery db "SELECT adjclose FROM googl" []
msftRaw <- quickQuery db "SELECT adjclose FROM msft" []
googl = map (fromSql . head) googlRaw :: [Double]
msft = map (fromSql . head) msftRaw :: [Double]

In [37]:
aapl252 = take 252 aapl

In [38]:
msft252 = take 252 msft

In [39]:
googl252 = take 252 googl

In [40]:
plot X11
    [ Data2D [Style Lines, Title "AAPL"] [] (zip [0,-1..] aapl252)
    , Data2D [Style Lines, Title "MSFT"] [] (zip [0,-1..] msft252)
    , Data2D [Style Lines, Title "GOOGL"] [] (zip [0,-1..] googl252)
    ]

True

In [41]:

rescale :: [Double] -> [Double]
rescale xs = map (\x -> (x - lead)/(diffxs)) xs
  where
    lead = last xs
    maxxs = maximum xs
    minxs = minimum xs
    diffxs = maxxs - minxs

In [43]:
plot X11
    [ Data2D [Style Lines, Title "AAPL"] [] (zip [0,-1..] (rescale aapl252))
    , Data2D [Style Lines, Title "MSFT"] [] (zip [0,-1..] (rescale msft252))
    , Data2D [Style Lines, Title "GOOGL"] [] (zip [0,-1..] (rescale googl252))
    ]

True